# 5. Generate Continuous Cycle


In [ ]:
# CELL 1 [TAG: parameters]
# ---------------------------------------------------------
# Default parameters (Airflow will OVERWRITE these)
# ---------------------------------------------------------
INPUT_TOPOLOGY_PATH = "s3://runs/test_run/topology.json"
MODEL_DIR = "s3://models/prod/"
REF_SEGMENTS_PATH = "s3://models/reference_segments.pkl"
OUTPUT_CYCLE_PATH = "s3://runs/test_run/cycle.csv"

# MinIO Credentials (DEFAULTS ONLY)
MINIO_ENDPOINT = "http://localhost:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password123"


In [ ]:
# CELL 2: Imports
import pickle
import json
import numpy as np
import pandas as pd
import s3fs


In [ ]:
# CELL 3: MinIO Configuration
# Initialize S3 Filesystem
fs = s3fs.S3FileSystem(
    key=MINIO_ACCESS_KEY,
    secret=MINIO_SECRET_KEY,
    client_kwargs={'endpoint_url': MINIO_ENDPOINT}
)

# Pandas storage options for saving CSV later
storage_options = {
    "key": MINIO_ACCESS_KEY,
    "secret": MINIO_SECRET_KEY,
    "client_kwargs": {"endpoint_url": MINIO_ENDPOINT}
}


In [ ]:
# CELL 4: Load Models & Topology
print("Loading artifacts from MinIO...")

# Ensure directory path format
model_base = MODEL_DIR.rstrip("/")

try:
    # 1. Load Transition Matrices
    with fs.open(f"{model_base}/transition_matrices.pkl", 'rb') as f:
        trans_matrices = pickle.load(f)

    # 2. Load State Definitions
    with fs.open(f"{model_base}/state_definitions.pkl", 'rb') as f:
        state_defs = pickle.load(f)

    # 3. Load Reference Segments
    with fs.open(REF_SEGMENTS_PATH, 'rb') as f:
        ref_segments = pickle.load(f)

    # 4. Load Topology (JSON)
    with fs.open(INPUT_TOPOLOGY_PATH, 'r') as f:
        topology = json.load(f)

    print("✅ All artifacts loaded successfully.")
except FileNotFoundError as e:
    print(f"❌ Missing Artifact: {e}")
    raise


In [ ]:
# CELL 5: Logic (Continuous Generation)

def generate_segment(duration, start_speed, matrix, states):
    # Find state index closest to the required start_speed
    start_node = np.argmin(np.abs(states[:, 0] - start_speed))
    
    current = start_node
    path = [current]
    
    for _ in range(duration - 1):
        probs = matrix[current]
        if np.sum(probs) == 0:
            next_state = current
        else:
            next_state = np.random.choice(len(probs), p=probs)
        
        path.append(next_state)
        current = next_state
        
    return states[path, 0]

final_cycle = []
handoff_speed = 0.0

print(f"Generating cycle for {len(topology)} topology blocks...")

for i, block in enumerate(topology):
    # 1. Determine Requested Group (0=Heavy, 1=Light)
    is_heavy = block['condition'] in ['SLOW', 'TRAFFIC_JAM']
    idx = 0 if is_heavy else 1
    
    # 2. Safety Check & Fallback
    # If the model for the requested group is missing (None), try the other group.
    if ref_segments[idx] is None or trans_matrices[idx] is None:
        original_idx = idx
        idx = 1 - idx # Toggle 0 -> 1 or 1 -> 0
        
        print(f"⚠️ Warning (Block {i}): Model for Group {original_idx} is missing.")
        
        if ref_segments[idx] is None:
            raise ValueError("❌ Critical Error: No models found for EITHER Heavy or Light traffic. Please check your Training Pipeline (Step 01/02).")
            
        print(f"   > Fallback: Using Group {idx} model instead.")

    # 3. Estimate Duration
    # Time = Distance / Average_Speed_of_Reference
    avg_speed_ms = np.mean(ref_segments[idx][:, 0]) / 3.6
    if avg_speed_ms < 1.0: avg_speed_ms = 1.0
    
    duration = int(block['length_m'] / avg_speed_ms)
    if duration < 5: duration = 5
    
    # 4. Generate
    segment = generate_segment(duration, handoff_speed, trans_matrices[idx], state_defs[idx])
    final_cycle.append(segment)
    
    handoff_speed = segment[-1]

# Stitch together
if final_cycle:
    final_speed_kmh = np.concatenate(final_cycle)
else:
    print("Warning: Empty cycle generated.")
    final_speed_kmh = np.array([])

print(f"Cycle construction complete. Total duration: {len(final_speed_kmh)}s")

In [ ]:
# CELL 6: Save Output
print(f"Saving generated cycle ({len(final_speed_kmh)} seconds) to {OUTPUT_CYCLE_PATH}...")
df = pd.DataFrame({'speed_kmh': final_speed_kmh})
df.to_csv(
    OUTPUT_CYCLE_PATH,
    index=False,
    storage_options=storage_options
)
print("✅ Cycle saved.")
